In [1]:
# Import the utility module
import notebook_utils

# Setup the path
notebook_utils.setup_path()

Added /home/amir/Codes/NN-dynamic-scaling to Python path


In [ ]:
# Now you can import from src
from src.models.mlp import MLP
from src.utils.cloning import clone_model
from src.utils import metrics

# Check the imported modules
model = MLP()
model

MLP(
  (layers): ModuleDict(
    (linear_0): Linear(in_features=784, out_features=512, bias=True)
    (act_0): ReLU()
    (linear_1): Linear(in_features=512, out_features=256, bias=True)
    (act_1): ReLU()
    (linear_2): Linear(in_features=256, out_features=128, bias=True)
    (act_2): ReLU()
    (out): Linear(in_features=128, out_features=10, bias=True)
  )
)

## Example: Using modules from src

In [ ]:
import torch
from src.models.mlp import MLP

# Create a simple MLP model
model = MLP(
    input_size=784,
    hidden_sizes=[128, 64],
    output_size=10,
    activation='relu'
)

print(model)

In [ ]:
# Generate random input
torch.manual_seed(42)
x = torch.randn(10, 784)

# Run forward pass
with torch.no_grad():
    output = model(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

## Accessing the new Gaussianity metrics

In [ ]:
from src.utils.metrics import measure_gaussianity

# Generate random activations
gaussian_activations = torch.randn(1000, 10)  # Normal distribution
uniform_activations = torch.rand(1000, 10)    # Uniform distribution

# Measure Gaussianity
gaussian_score = measure_gaussianity(gaussian_activations, method="shapiro")
uniform_score = measure_gaussianity(uniform_activations, method="shapiro")

print(f"Gaussianity score for normal distribution: {gaussian_score:.4f}")
print(f"Gaussianity score for uniform distribution: {uniform_score:.4f}")
print(f"Uniform distribution is {uniform_score/gaussian_score:.1f}x less Gaussian")

## Reloading modules after changes

If you make changes to files in the `src` directory while working in the notebook, you can reload the modules to pick up those changes.

In [ ]:
# After making changes to src/utils/metrics.py
import importlib
from src.utils import metrics

# Reload the module
metrics = importlib.reload(metrics)

# Or if using the utility module:
# metrics = notebook_utils.reload_module(metrics)